# Test in Centro Hospitalar Universitário de São João's (CHUSP) Dataset

### Using the models from **Run076** (IRF), **Run111** (SRF), and **Run114** (PED) in Binary Implementation and model from **Run058** in Multi-class

### Multi-class Model

#### Resized Images

In [ ]:
from test_model import test_model

test_model(
    fold_test=1,
    model_name="UNet",
    weights_name="Run058_UNet_best_model.pth",
    number_of_channels=1,
    number_of_classes=4,
    dataset="chusj",
    device_name="GPU",
    batch_size=1,
    patch_type="vertical",
    save_images=True,
    resize_images=True,
    final_test=True
)

#### Original Images

In [ ]:
from test_model import test_model

test_model(
    fold_test=1,
    model_name="UNet",
    weights_name="Run058_UNet_best_model.pth",
    number_of_channels=1,
    number_of_classes=4,
    dataset="chusj",
    device_name="GPU",
    batch_size=1,
    patch_type="vertical",
    save_images=True,
    resize_images=False,
    final_test=True
)

### Binary Models

#### Resized Images

In [ ]:
from test_model import test_model

test_model(
    fold_test=1,
    model_name="UNet3",
    weights_name="Run076111114_UNet3",
    number_of_channels=1,
    number_of_classes=2,
    dataset="chusj",
    device_name="GPU",
    batch_size=1,
    patch_type="vertical",
    save_images=True,
    resize_images=True,
    final_test=True,
    selected_models=[76,111,114],
    mode='highest_prob'
)

#### Original Images

In [ ]:
from test_model import test_model

test_model(
    fold_test=1,
    model_name="UNet3",
    weights_name="Run076111114_UNet3",
    number_of_channels=1,
    number_of_classes=2,
    dataset="chusj",
    device_name="GPU",
    batch_size=1,
    patch_type="vertical",
    save_images=True,
    resize_images=False,
    final_test=True,
    selected_models=[76,111,114],
    mode='highest_prob'
)

### Calculate Dice - Processed Images

The Dice coefficient is calculated for the same slices after being processed. The path to the processed images is informed the results are saved.

Declare the path to the processed images below:

In [ ]:
processed_folder = ""
resized = True
binary = False


if binary:
    run_name = "Run076111114"
else:
    run_name = "Run058"

if resized:
    gt_masks_folder = "D:\DavidTerroso\Images\OCT_images\chusj\masks_resized"
    oct_folder = "D:\DavidTerroso\Images\OCT_images\chusj\slices_resized" 
else:
    gt_masks_folder = "D:\DavidTerroso\Images\OCT_images\chusj\masks"
    oct_folder = "D:\DavidTerroso\Images\OCT_images\chusj\slices" 

In [ ]:
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import numpy as np
import torch
from os import listdir, makedirs
from os.path import exists
from pandas import DataFrame, Series
from shutil import rmtree
from skimage.io import imread
from tqdm import tqdm_notebook as tqdm
from networks.loss import dice_coefficient
from paths import IMAGES_PATH

slice_results = []

# Dictionary of labels in masks to fluid names
label_to_fluids = {
    0: "Background",
    1: "IRF",
    2: "SRF",
    3: "PED"
}

# Dictionary of fluid to labels in masks
fluids_to_label = {
    "IRF": 1,
    "SRF": 2,
    "PED": 3
} 

if not resized: 
    folder_to_save = IMAGES_PATH + f"\\OCT_images\\segmentation\\predictions\\{run_name}_final_processed_chusj\\"
    folder_to_save_masks = IMAGES_PATH + f"\\OCT_images\\segmentation\\predictions\\{run_name}_final_processed_chusj_masks\\"
    folder_to_save_gts = IMAGES_PATH + f"\\OCT_images\\segmentation\\predictions\\{run_name}_final_processed_chusj_gts\\"

else:
    folder_to_save = IMAGES_PATH + f"\\OCT_images\\segmentation\\predictions\\{run_name}_resized_final_processed_chusj\\"
    folder_to_save_masks = IMAGES_PATH + f"\\OCT_images\\segmentation\\predictions\\{run_name}_resized_final_processed_chusj\\"
    folder_to_save_gts = IMAGES_PATH + f"\\OCT_images\\segmentation\\predictions\\{run_name}_resized_final_processed_chusj_gts\\"
    
    # In case the folder to save exists, it is deleted and created again
    if exists(folder_to_save):
        rmtree(folder_to_save)
        makedirs(folder_to_save)
    else:
        makedirs(folder_to_save)
    if exists(folder_to_save_masks):
        rmtree(folder_to_save_masks)
        makedirs(folder_to_save_masks)
    else:
        makedirs(folder_to_save_masks)
    if exists(folder_to_save_gts):
        rmtree(folder_to_save_gts)
        makedirs(folder_to_save_gts)
    else:
        makedirs(folder_to_save_gts)


with tqdm(listdir(processed_folder), total=len(listdir(processed_folder)), desc='Testing Model', unit='img', leave=True, position=0) as progress_bar:
    for img_path in listdir(processed_folder):
        image_name = img_path.split("\\")[-1][:-5]
        oct_image = imread(str(oct_folder + "\\" + image_name + ".tiff"))
        pred = torch.from_numpy(imread(img_path))
        gt = torch.from_numpy(imread(str(gt_masks_folder + "\\" + image_name + "_predicted.tiff")))
        dice_scores, voxel_counts, union_counts, intersection_counts, binary_dice = dice_coefficient(model_name="UNet", target=gt, prediction=pred, num_classes=4)
        slice_results.append([image_name, *dice_scores, *voxel_counts, *union_counts, *intersection_counts, binary_dice])

        # Declares the name under which the masks will be saved and writes the path to the original B-scan
        predicted_mask_name = folder_to_save + image_name + "_predicted.tiff"

        # Saves the segmentation masks
        pred = np.array(pred.cpu().numpy(), dtype=np.float32)[0]

        # Converts each voxel classified as background to 
        # NaN so that it will not appear in the overlaying
        # mask
        pred[pred == 0] = np.nan

        # The voxels classified in "IRF", "SRF", and "PED" 
        # will be converted to color as Red for IRF, green 
        # for SRF, and blue for PED
        fluid_colors = ["red", "green", "blue"]
        fluid_cmap = mcolors.ListedColormap(fluid_colors)
        # Declares in which part of the color bar each
        # label is going to be placed
        fluid_bounds = [1, 2, 3, 4]
        # Normalizes the color map according to the 
        # bounds declared.
        fluid_norm = mcolors.BoundaryNorm(fluid_bounds, fluid_cmap.N)

        # Saves the OCT scan with an overlay of the predicted masks
        plt.figure(figsize=(oct_image.shape[1] / 100, oct_image.shape[0] / 100))
        plt.imshow(oct_image, cmap=plt.cm.gray)
        plt.imshow(pred, alpha=0.3, cmap=fluid_cmap, norm=fluid_norm)
        plt.axis("off")
        plt.savefig(predicted_mask_name, bbox_inches='tight', pad_inches=0)

        # Closes the figure
        plt.clf()
        plt.close("all")

        progress_bar.update(1)

slice_df = DataFrame(slice_results, 
                    columns=["slice", 
                            *[f"dice_{label_to_fluids.get(i)}" for i in range(4)], 
                            *[f"voxels_{label_to_fluids.get(i)}" for i in range(4)], 
                            *[f"union_{label_to_fluids.get(i)}" for i in range(4)], 
                            *[f"intersection_{label_to_fluids.get(i)}" for i in range(4)],
                            "binary_dice"])
if not resized:
    slice_df.to_csv(f"results/{run_name}_slice_dice_final_processed_chusj.csv", index=False)
else:
    slice_df.to_csv(f"results/{run_name}_slice_dice_resized_final_processed_chusj.csv", index=False)

# Adds the vendor, volume, and number of the slice information to the DataFrame
slice_df[['vendor', 'volume', 'slice_number']] = slice_df['slice'].str.replace('.tiff', '', regex=True).str.split('_', n=2, expand=True)
# Saves the DataFrame with the mean and standard deviation for each OCT volume (e.g. mean (standard deviation))
volume_df_mean = slice_df[["volume", "dice_IRF", "dice_SRF", "dice_PED"]].groupby("volume").mean()
volume_df_mean.index.name = "Volume"
volume_df_std = slice_df[["volume", "dice_IRF", "dice_SRF", "dice_PED"]].groupby("volume").std()
volume_df_std.index.name = "Volume"
resulting_volume_df = volume_df_mean.astype(str) + " (" + volume_df_std.astype(str) + ")"

# Saves the DataFrame with the mean and standard deviation for each vendor (e.g. mean (standard deviation))
vendor_df_mean = slice_df[["vendor", "dice_IRF", "dice_SRF", "dice_PED"]].groupby("vendor").mean()
vendor_df_mean.index.name = "Vendor"
vendor_df_std = slice_df[["vendor", "dice_IRF", "dice_SRF", "dice_PED"]].groupby("vendor").std()
vendor_df_std.index.name = "Vendor"
resulting_vendor_df = vendor_df_mean.astype(str) + " (" + vendor_df_std.astype(str) + ")"

# Saves the DataFrame with the mean and standard deviation for each class (e.g. mean (standard deviation))
class_df_mean = slice_df[["dice_IRF", "dice_SRF", "dice_PED"]].mean().to_frame().T
class_df_std = slice_df[["dice_IRF", "dice_SRF", "dice_PED"]].std().to_frame().T
resulting_class_df = class_df_mean.astype(str) + " (" + class_df_std.astype(str) + ")"

# Saves the DataFrame that contains the values for each volume, class, and vendor
if not resized:
    resulting_volume_df.to_csv(f"results/{run_name}_volume_dice_final_chusj.csv", index=True)
    resulting_class_df.to_csv(f"results/{run_name}_class_dice_final_chusj.csv", index=False)
    resulting_vendor_df.to_csv(f"results/{run_name}_vendor_dice_final_chusj.csv", index=True)            
else:
    resulting_volume_df.to_csv(f"results/{run_name}_volume_dice_resized_final_chusj.csv", index=True)
    resulting_class_df.to_csv(f"results/{run_name}_class_dice_resized_final_chusj.csv", index=False)
    resulting_vendor_df.to_csv(f"results/{run_name}_vendor_dice_resized_final_chusj.csv", index=True)

# Handles the information only on the slices that have the fluid
slice_df_wf = slice_df.copy()
# Iterates through all the classes available
for i in range(4):
    # Sets the Dice values to NaN whenever there is no fluid of that type
    slice_df_wf.loc[slice_df_wf[f"voxels_{label_to_fluids.get(i)}"] == 0, f"dice_{label_to_fluids.get(i)}"] = np.nan

# Adds the vendor, volume, and number of the slice information to the DataFrame
slice_df_wf[['vendor', 'volume', 'slice_number']] = slice_df_wf['slice'].str.replace('.tiff', '', regex=True).str.split('_', n=2, expand=True)
# Saves the DataFrame with the mean and standard deviation for each OCT volume (e.g. mean (standard deviation))
volume_df_mean = slice_df_wf[["volume", "dice_IRF", "dice_SRF", "dice_PED"]].groupby("volume").mean()
volume_df_mean.index.name = "Volume"
volume_df_std = slice_df_wf[["volume", "dice_IRF", "dice_SRF", "dice_PED"]].groupby("volume").std()
volume_df_std.index.name = "Volume"
resulting_volume_df = volume_df_mean.astype(str) + " (" + volume_df_std.astype(str) + ")"

# Saves the DataFrame with the mean and standard deviation for each vendor (e.g. mean (standard deviation))
vendor_df_mean = slice_df_wf[["vendor", "dice_IRF", "dice_SRF", "dice_PED"]].groupby("vendor").mean()
vendor_df_mean.index.name = "Vendor"
vendor_df_std = slice_df_wf[["vendor", "dice_IRF", "dice_SRF", "dice_PED"]].groupby("vendor").std()
vendor_df_std.index.name = "Vendor"
resulting_vendor_df = vendor_df_mean.astype(str) + " (" + vendor_df_std.astype(str) + ")"

# Saves the DataFrame with the mean and standard deviation for each class (e.g. mean (standard deviation))
class_df_mean = slice_df_wf[["dice_IRF", "dice_SRF", "dice_PED"]].mean().to_frame().T
class_df_std = slice_df_wf[["dice_IRF", "dice_SRF", "dice_PED"]].std().to_frame().T
resulting_class_df = class_df_mean.astype(str) + " (" + class_df_std.astype(str) + ")"

# Saves the DataFrame that contains the values for each volume, class, and vendor
if not resized:
    resulting_volume_df.to_csv(f"results/{run_name}_volume_dice_wfluid_final_processed_chusj.csv", index=True)
    resulting_class_df.to_csv(f"results/{run_name}_class_dice_wfluid_final_processed_chusj.csv", index=False)
    resulting_vendor_df.to_csv(f"results/{run_name}_vendor_dice_wfluid_final_processed_chusj.csv", index=True)
else:
    resulting_volume_df.to_csv(f"results/{run_name}_volume_dice_resized_wfluid_final_processed_chusj.csv", index=True)
    resulting_class_df.to_csv(f"results/{run_name}_class_dice_resized_wfluid_final_processed_chusj.csv", index=False)
    resulting_vendor_df.to_csv(f"results/{run_name}_vendor_dice_resized_wfluid_final_processed_chusj.csv", index=True)

# Handles the information only on the slices that do not have fluid
slice_df_wof = slice_df.copy()
# Iterates through all the classes available
for i in range(4):
    # Gets the DataFrame that contains a non-negative number of voxels for each column
    slice_df_wof.loc[slice_df_wof[f"voxels_{label_to_fluids.get(i)}"] > 0, f"dice_{label_to_fluids.get(i)}"] = np.nan

# Adds the vendor, volume, and number of the slice information to the DataFrame
slice_df_wof[['vendor', 'volume', 'slice_number']] = slice_df_wof['slice'].str.replace('.tiff', '', regex=True).str.split('_', n=2, expand=True)
# Saves the DataFrame with the mean and standard deviation for each OCT volume (e.g. mean (standard deviation))
volume_df_mean = slice_df_wof[["volume", "dice_IRF", "dice_SRF", "dice_PED"]].groupby("volume").mean()
volume_df_mean.index.name = "Volume"
volume_df_std = slice_df_wof[["volume", "dice_IRF", "dice_SRF", "dice_PED"]].groupby("volume").std()
volume_df_std.index.name = "Volume"
resulting_volume_df = volume_df_mean.astype(str) + " (" + volume_df_std.astype(str) + ")"

# Saves the DataFrame with the mean and standard deviation for each vendor (e.g. mean (standard deviation))
vendor_df_mean = slice_df_wof[["vendor", "dice_IRF", "dice_SRF", "dice_PED"]].groupby("vendor").mean()
vendor_df_mean.index.name = "Vendor"
vendor_df_std = slice_df_wof[["vendor", "dice_IRF", "dice_SRF", "dice_PED"]].groupby("vendor").std()
vendor_df_std.index.name = "Vendor"
resulting_vendor_df = vendor_df_mean.astype(str) + " (" + vendor_df_std.astype(str) + ")"

# Saves the DataFrame with the mean and standard deviation for each class (e.g. mean (standard deviation))
class_df_mean = slice_df_wof[["dice_IRF", "dice_SRF", "dice_PED"]].mean().to_frame().T
class_df_std = slice_df_wof[["dice_IRF", "dice_SRF", "dice_PED"]].std().to_frame().T
resulting_class_df = class_df_mean.astype(str) + " (" + class_df_std.astype(str) + ")"

# Saves the DataFrame that contains the values for each volume, class, and vendor
if not resized:
    resulting_volume_df.to_csv(f"results/{run_name}_volume_dice_wofluid_final_processed_chusj.csv", index=True)
    resulting_class_df.to_csv(f"results/{run_name}_class_dice_wofluid_final_processed_chusj.csv", index=False)
    resulting_vendor_df.to_csv(f"results/{run_name}_vendor_dice_wofluid_final_processed_chusj.csv", index=True)
    binary_dices_name = f"fluid_dice_final__processed_chusj"
else:
    resulting_volume_df.to_csv(f"results/{run_name}_volume_dice_resized_wofluid_final_processed_chusj.csv", index=True)
    resulting_class_df.to_csv(f"results/{run_name}_class_dice_resized_wofluid_final_processed_chusj.csv", index=False)
    resulting_vendor_df.to_csv(f"results/{run_name}_vendor_dice_resized_wofluid_final_processed_chusj.csv", index=True)
    binary_dices_name = f"fluid_dice_resized_final_processed_chusj"

# Initializes a list that will hold the results for the binary case
binary_dices = []
# Appends the binary Dice coefficient to a list that holds these values
binary_dices.append(f"{slice_df['binary_dice'].mean()} ({slice_df['binary_dice'].std()})")

# Get the fluid voxel column names (i = 1, 2, 3)
fluid_voxel_cols = [f"voxels_{label_to_fluids[i]}" for i in [1, 2, 3]]

# Copies the original DataFrame 
# on which changes will be applied
slice_df_wf = slice_df.copy()
slice_df_wof = slice_df.copy()

# For the 'with fluid' DataFrame, the slices with no fluid will be set to NaN in the 
# column that contains the binary Dice        
slice_df_wf.loc[slice_df_wf[fluid_voxel_cols].sum(axis=1) == 0, 'binary_dice'] = np.nan

# For the 'without fluid' DataFrame, the slices with any fluid will be set to NaN in the 
# column that contains the binary Dice
slice_df_wof.loc[slice_df_wof[fluid_voxel_cols].sum(axis=1) > 0, 'binary_dice'] = np.nan

# The mean and std of each column is added to a list that contains the results in binary conditions
binary_dices.append(f"{slice_df_wf['binary_dice'].mean()} ({slice_df_wf['binary_dice'].std()})")
binary_dices.append(f"{slice_df_wof['binary_dice'].mean()} ({slice_df_wof['binary_dice'].std()})")

# Saves the results as a DataFrame
df = Series(binary_dices).to_frame().T
df.columns = ["AllSlices", "SlicesWithFluid", "SlicesWithoutFluid"]
df.to_csv(f"results/{run_name}_{binary_dices_name}.csv", index=False)